In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import os 
import seaborn as sns
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  RobustScaler
import pickle
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from keras import regularizers, layers, optimizers, initializers

from tensorflow.keras.applications import EfficientNetV2M
import numpy as np
import gc

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(f'Current GPU allocator: {os.getenv("TF_GPU_ALLOCATOR")}')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            print(f'Setting memory growth for {gpu}')
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

    

2024-04-17 18:25:24.990472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 18:25:25.615565: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Current GPU allocator: cuda_malloc_async
Setting memory growth for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-04-17 18:25:26.177901: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-17 18:25:26.202337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-17 18:25:26.202407: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
study_name = '417_nas_all_3'

In [3]:

mean_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']


selected_features_pickle_path = './data/selected_features_list.pickle'
with open(selected_features_pickle_path, 'rb') as f:
    FEATURE_COLS = pickle.load(f)


In [4]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [5]:
sd_columns = [col for col in train_df.columns if col.endswith('_sd')]
train_df.drop(columns=sd_columns, inplace=True)

In [6]:
train_images_path = './data/train_images/'
test_images_path = './data/test_images/'    

train_df['image_path'] = train_df['id'].apply(lambda x: os.path.join(train_images_path, f'{x}.jpeg'))
test_df['image_path'] = test_df['id'].apply(lambda x: os.path.join(test_images_path, f'{x}.jpeg'))

In [7]:
for column in mean_columns:
    lower_quantile = train_df[column].quantile(0.005)
    upper_quantile = train_df[column].quantile(0.985)  
    train_df = train_df[(train_df[column] >= lower_quantile) & (train_df[column] <= upper_quantile)]

In [8]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, trait in enumerate(mean_columns):

    # Determine the bin edges dynamically based on the distribution of traits
    bin_edges = np.percentile(train_df[trait], np.linspace(0, 100, 5 + 1))
    train_df[f"bin_{i}"] = np.digitize(train_df[trait], bin_edges)

# Concatenate the bins into a final bin
train_df["final_bin"] = (
    train_df[[f"bin_{i}" for i in range(len(mean_columns))]]
    .astype(str)
    .agg("".join, axis=1)
)

# Perform the stratified split using final bin
train_df = train_df.reset_index(drop=True)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df, train_df["final_bin"])):
    train_df.loc[valid_idx, "fold"] = fold

/home/tobias/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [9]:
scaler = RobustScaler()

sample_df = train_df.copy()
train_df = sample_df[sample_df.fold != 3]
valid_df = sample_df[sample_df.fold == 3]
print(f"# Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")


train_df[FEATURE_COLS] = scaler.fit_transform(train_df[FEATURE_COLS].values)
valid_df[FEATURE_COLS] = scaler.transform(valid_df[FEATURE_COLS].values)

# Num Train: 39335 | Num Valid: 9833


/tmp/ipykernel_1026380/2079233736.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[FEATURE_COLS] = scaler.fit_transform(train_df[FEATURE_COLS].values)
/tmp/ipykernel_1026380/2079233736.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df[FEATURE_COLS] = scaler.transform(valid_df[FEATURE_COLS].values)


In [10]:
import glob

directory_path = './NN_search'
pattern = f"{directory_path}/{study_name}_best_val_*.h5"

files = glob.glob(pattern)

max_r2_score = float('-inf')
best_model = None

# Käy läpi jokainen tiedosto ja etsi suurin r2_score_inv
for file in files:
    value = float(file.split('best_val')[1].split('_')[1])
    if value > max_r2_score:
        max_r2_score = value
        best_model = file


# Tulosta suurin löydetty r2_score_inv ja vastaava tiedosto
print(f"Best R2-score: {max_r2_score:.5f}")
if best_model:
    print(f"Best model: {best_model}")
else:
    print("No best model found")

best_log_transforms_name =  f'./NN_search/{study_name}_{max_r2_score:.5f}_best_log_transforms.pickle'
best_scalers_name = f'./NN_search/{study_name}_{max_r2_score:.5f}_best_scalers.pickle'

print(f'Opening log transforms from {best_log_transforms_name}')
with open(best_log_transforms_name, 'rb') as f:
    log_transforms = pickle.load(f)

print(f'Opening scalers from {best_scalers_name}')
with open(best_scalers_name, 'rb') as f:
    scaler_transforms = pickle.load(f)
        

Best R2-score: 0.29580
Best model: ./NN_search/417_nas_all_3_best_val_0.29580_model.h5
Opening log transforms from ./NN_search/417_nas_all_3_0.29580_best_log_transforms.pickle
Opening scalers from ./NN_search/417_nas_all_3_0.29580_best_scalers.pickle


In [ ]:
print(f'Scaler are: {scaler_transforms}')
print(f'Log transforms are: {log_transforms}')

In [11]:
def augment_image(img):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, lower=0.5, upper=1.5)    
    img = tf.image.random_crop(img, size=[480, 480, 3])  
    return img


def process_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    img = augment_image(img)  
    return img

def process_image_valid(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    return img



# Define your dataset processing function
def process_path_train(file_path, tabular_data, targets):
    img = process_image(file_path)
    return (img, img, tabular_data), targets


def process_path_valid(file_path, tabular_data, targets):
    img = process_image_valid(file_path)
    return (img, img, tabular_data), targets

In [12]:
y_train = train_df[mean_columns]
y_valid = valid_df[mean_columns]


y_train_transformed = y_train.copy()
y_valid_transformed = y_valid.copy()


for target, log_base in log_transforms.items():
    if log_base is not None and log_base != 'sqrt' and log_base != 'cbrt':
        y_train_transformed[target] = np.log(y_train[target]) / np.log(log_base)
        y_valid_transformed[target] = np.log(y_valid[target]) / np.log(log_base)

    elif log_base == 'sqrt':
        y_train_transformed[target] = np.sqrt(y_train[target])
        y_valid_transformed[target] = np.sqrt(y_valid[target])

    elif log_base == 'cbrt':
        y_train_transformed[target] = np.cbrt(y_train[target])
        y_valid_transformed[target] = np.cbrt(y_valid[target])

    else:
        y_train_transformed[target] = y_train[target]
        y_valid_transformed[target] = y_valid[target]    

for target, scaler in scaler_transforms.items():
    if scaler is not None:
        y_train_transformed[target] = scaler.fit_transform(y_train_transformed[target].values.reshape(-1, 1)).flatten()
        y_valid_transformed[target] = scaler.transform(y_valid_transformed[target].values.reshape(-1, 1)).flatten()


In [13]:
BUFFER_SIZE = 50000 

train_tabular = train_df[FEATURE_COLS].values
valid_tabular = valid_df[FEATURE_COLS].values

train_images_path = train_df['image_path'].values
valid_images_path = valid_df['image_path'].values

train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path, train_tabular, y_train_transformed.values))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images_path, valid_tabular, y_valid_transformed.values))

train_dataset = train_dataset.map(process_path_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)

BATCH_SIZE = 64

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)






2024-04-17 18:25:28.068285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-17 18:25:28.068399: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-17 18:25:28.068449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-17 18:25:28.183716: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-17 18:25:28.183794: I tensorflow/compile

In [14]:
from tensorflow.keras.callbacks import TensorBoard


tf.keras.backend.clear_session()
gc.collect()


# Aseta logitiedostojen hakemisto
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1, update_freq='epoch')


import tensorflow as tf
import numpy as np

class EpochBasedLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, initial_learning_rate, warmup_epochs, warmup_target, total_epochs, alpha):
        super(EpochBasedLRSchedule, self).__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_epochs = warmup_epochs
        self.warmup_target = warmup_target
        self.total_epochs = total_epochs
        self.alpha = alpha
        self.current_epoch = tf.Variable(0, dtype=tf.int64, trainable=False)

    def on_epoch_begin(self, epoch):
        self.current_epoch.assign(epoch)

    def __call__(self, step):
        epoch = tf.cast(self.current_epoch, tf.float32)
        linear_warmup = self.initial_learning_rate + (epoch / self.warmup_epochs) * (self.warmup_target - self.initial_learning_rate)
        cosine_decay = self.alpha + (self.warmup_target - self.alpha) * 0.5 * (1 + tf.cos(tf.constant(np.pi, dtype=tf.float32) * (epoch - self.warmup_epochs) / (self.total_epochs - self.warmup_epochs)))
        learning_rate = tf.where(epoch < self.warmup_epochs, linear_warmup, cosine_decay)
        return learning_rate
    
    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "warmup_epochs": self.warmup_epochs,
            "warmup_target": self.warmup_target,
            "total_epochs": self.total_epochs,
            "alpha": self.alpha
        }

# Käytä mukautettua oppimisnopeuden aikataulua
lr_schedule_custom = EpochBasedLRSchedule(
    initial_learning_rate=0.0001,
    warmup_epochs=3,
    warmup_target=0.0005,
    total_epochs=20,
    alpha=0.00001
)

class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Haetaan nykyinen oppimisnopeus optimizerilta
        lr = self.model.optimizer._decayed_lr(tf.float32).numpy()
        print(f"Learning rate at epoch {epoch+1} is {lr}")

class UpdateLRScheduleCallback(tf.keras.callbacks.Callback):
    def __init__(self, lr_schedule):
        self.lr_schedule = lr_schedule

    def on_epoch_begin(self, epoch, logs=None):
        self.lr_schedule.on_epoch_begin(epoch)

# Luo callback-olio
update_lr_schedule = UpdateLRScheduleCallback(lr_schedule_custom)


def r2_score_tf(y_true, y_pred):

    try: 
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
        r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
        return tf.reduce_mean(tf.maximum(r2, 0.0))
    except Exception as e:
        # print(f'Error in r2_score_tf: {e}')
        return float('-inf')

image_input_avg = Input(shape=(480, 480, 3), name='image_input_avg')
image_input_max = Input(shape=(480, 480, 3), name='image_input_max')
tabular_input = Input(shape=(len(FEATURE_COLS),), name='tabular_input')

eff_avg = EfficientNetV2M(weights='imagenet', include_top=False, pooling='avg', input_tensor=image_input_avg)

eff_avg.trainable = True 
for layer in eff_avg.layers[:-4]:
    layer.trainable = False

eff_avg_output = eff_avg(image_input_avg)

model_avg  = Model(inputs=image_input_avg, outputs = eff_avg_output, name='model_avg')

eff_max = EfficientNetV2M(weights='imagenet', include_top=False, pooling='max', input_tensor=image_input_max)

for layer in eff_max.layers[:-4]:
    layer.trainable = False

eff_max_output = eff_max(image_input_max)

model_max = Model(inputs=image_input_max, outputs = eff_max_output, name='model_max')

model_avg = model_avg(image_input_avg)
model_max = model_max(image_input_max)

concatenated_img = Concatenate()([model_avg, model_max])

img_layer = Dense(396, activation='LeakyReLU', kernel_initializer = 'glorot_uniform')(concatenated_img)
img_layer = layers.BatchNormalization()(img_layer)
img_layer = Dropout(0.7)(img_layer)

tab_layer = Dense(419, activation='relu', kernel_initializer = 'random_normal')(tabular_input)
tab_layer = layers.BatchNormalization()(tab_layer)
tab_layer = Dropout(0.2)(tab_layer)

concatanate_all = Concatenate()([img_layer, tab_layer])

dense_layer = Dense(378, activation='elu', kernel_initializer = 'lecun_uniform')(concatanate_all)
dense_layer = Dropout(0.1)(dense_layer)

output_layer = Dense(6, activation='linear')(dense_layer)

model = Model(inputs=[image_input_avg, image_input_max, tabular_input], outputs=output_layer)

model.compile(optimizer= optimizers.RMSprop(learning_rate = lr_schedule_custom), loss='mse', metrics=['mse', 'mae', r2_score_tf])

model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input_avg (InputLayer)   [(None, 480, 480, 3  0           []                               
                                )]                                                                
                                                                                                  
 image_input_max (InputLayer)   [(None, 480, 480, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_avg (Functional)         (None, 1280)         53150388    ['image_input_avg[0][0]']        
                                                                                              

In [15]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=f'./NN_search/testifinetus_best_val.h5', monitor='val_r2_score_tf', save_best_only=True, mode = 'max',  verbose = 1),
    LearningRateLogger(),
    tensorboard_callback,
    update_lr_schedule
]

history = model.fit(train_dataset, validation_data=valid_dataset, epochs=20, verbose=1, callbacks=callbacks)
# model.load_weights(f'./NN_search/testifinetus_best_val.h5')
from tensoflor.keras.models import load_model
model = load_model(f'./NN_search/testifinetus_best_val.h5', custom_objects={'r2_score_tf': r2_score_tf})

Epoch 1/20


2024-04-17 18:25:43.917405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [39335,6]
	 [[{{node Placeholder/_2}}]]
2024-04-17 18:25:43.917714: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [39335,81]
	 [[{{node Placeholder/_1}}]]
2024-04-17 18:26:06.944777: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/model_avg/efficientnetv2-m/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-04-17 1

615/615 [==============================] - ETA: 0s - loss: 1.2794 - mse: 1.2794 - mae: 0.8621 - r2_score_tf: 0.0097

2024-04-17 18:37:37.272428: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [9833]
	 [[{{node Placeholder/_0}}]]
2024-04-17 18:37:37.272724: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [9833,6]
	 [[{{node Placeholder/_2}}]]



Epoch 1: val_r2_score_tf improved from -inf to 0.11837, saving model to ./NN_search/testifinetus_best_val.h5
Learning rate at epoch 0 is 9.999999747378752e-05
615/615 [==============================] - 898s 1s/step - loss: 1.2794 - mse: 1.2794 - mae: 0.8621 - r2_score_tf: 0.0097 - val_loss: 0.5848 - val_mse: 0.5848 - val_mae: 0.5679 - val_r2_score_tf: 0.1184
Epoch 2/20
615/615 [==============================] - ETA: 0s - loss: 0.7790 - mse: 0.7790 - mae: 0.6700 - r2_score_tf: 0.0406
Epoch 2: val_r2_score_tf improved from 0.11837 to 0.13142, saving model to ./NN_search/testifinetus_best_val.h5
Learning rate at epoch 1 is 0.00023333332501351833
615/615 [==============================] - 859s 1s/step - loss: 0.7790 - mse: 0.7790 - mae: 0.6700 - r2_score_tf: 0.0406 - val_loss: 0.5113 - val_mse: 0.5113 - val_mae: 0.5167 - val_r2_score_tf: 0.1314
Epoch 3/20
615/615 [==============================] - ETA: 0s - loss: 0.5966 - mse: 0.5966 - mae: 0.5722 - r2_score_tf: 0.0895
Epoch 3: val_r2_sco

In [27]:

class EpochBasedLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, initial_learning_rate, warmup_epochs, warmup_target, total_epochs, alpha):
        super(EpochBasedLRSchedule, self).__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_epochs = warmup_epochs
        self.warmup_target = warmup_target
        self.total_epochs = total_epochs
        self.alpha = alpha
        self.current_epoch = tf.Variable(0, dtype=tf.int64, trainable=False)

    def on_epoch_begin(self, epoch):
        self.current_epoch.assign(epoch)

    def __call__(self, step):
        epoch = tf.cast(self.current_epoch, tf.float32)
        linear_warmup = self.initial_learning_rate + (epoch / self.warmup_epochs) * (self.warmup_target - self.initial_learning_rate)
        cosine_decay = self.alpha + (self.warmup_target - self.alpha) * 0.5 * (1 + tf.cos(tf.constant(np.pi, dtype=tf.float32) * (epoch - self.warmup_epochs) / (self.total_epochs - self.warmup_epochs)))
        learning_rate = tf.where(epoch < self.warmup_epochs, linear_warmup, cosine_decay)
        return learning_rate
    
    def get_config(self):
        super(EpochBasedLRSchedule, self)
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "warmup_epochs": self.warmup_epochs,
            "warmup_target": self.warmup_target,
            "total_epochs": self.total_epochs,
            "alpha": self.alpha
        }

model.summary()
model.save(f'./NN_search/finetunetest1_model.h5')


NotImplementedError: Learning rate schedule 'EpochBasedLRSchedule' must override `get_config()` in order to be serializable.

In [19]:
train_tabular = train_df[FEATURE_COLS].values
valid_tabular = valid_df[FEATURE_COLS].values

train_images_path = train_df['image_path'].values
valid_images_path = valid_df['image_path'].values

train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path, train_tabular, y_train_transformed.values))
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images_path, valid_tabular, y_valid_transformed.values))

train_dataset = train_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)

BATCH_SIZE = 64

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)




import numpy as np
import tensorflow as tf
import optuna
from keras import regularizers, layers, optimizers, initializers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from datetime import timedelta
import time
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, QuantileTransformer, RobustScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

## TEST DATA TEST

tf.keras.backend.clear_session()
gc.collect()

train_pred = model.predict(train_dataset, verbose=1)

for i, target in enumerate(mean_columns):
    print(f'Scaler transforming target : {target} with scaler : {scaler_transforms[target]}')
    scaler = scaler_transforms[target]
    if scaler is not None:
        train_pred[:, i] = scaler.inverse_transform(train_pred[:, i].reshape(-1, 1)).flatten()


for i, target in enumerate(mean_columns):
    print(f'Logpot transforming target : {target}, log transform : {log_transforms[target]}')
    log_base = log_transforms[target]
    if log_base is not None and log_base != 'sqrt' and log_base != 'cbrt':
        train_pred[:, i] = np.power(log_base, train_pred[:, i])
    elif log_base == 'sqrt':
        train_pred[:, i] = np.square(train_pred[:, i])
    elif log_base == 'cbrt':
        train_pred[:, i] = np.power(train_pred[:, i], 3)

R2_train = r2_score(y_train, train_pred)
MSE_train = mean_squared_error(y_train, train_pred)
MAE_train = mean_absolute_error(y_train, train_pred)

print(f'Train scores:\nR2 : {R2_train:.5f}, MSE : {MSE_train:.5f}, MAE : {MAE_train:.5f}')


2024-04-17 23:34:30.949720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [39335]
	 [[{{node Placeholder/_0}}]]
2024-04-17 23:34:30.950086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [39335,6]
	 [[{{node Placeholder/_2}}]]


615/615 [==============================] - 645s 1s/step
Scaler transforming target : X4_mean with scaler : PowerTransformer()
Scaler transforming target : X11_mean with scaler : StandardScaler()
Scaler transforming target : X18_mean with scaler : None
Scaler transforming target : X50_mean with scaler : None
Scaler transforming target : X26_mean with scaler : MinMaxScaler()
Scaler transforming target : X3112_mean with scaler : None
Logpot transforming target : X4_mean, log transform : 5
Logpot transforming target : X11_mean, log transform : 15
Logpot transforming target : X18_mean, log transform : sqrt
Logpot transforming target : X50_mean, log transform : None
Logpot transforming target : X26_mean, log transform : cbrt
Logpot transforming target : X3112_mean, log transform : 15
Train scores:
R2 : 0.43560, MSE : 576052.93545, MAE : 160.91438


In [23]:

## VALIDATION DATA TEST

tf.keras.backend.clear_session()
gc.collect()

valid_pred = model.predict(valid_dataset, verbose=1)

for i, target in enumerate(mean_columns):
    print(f'Scaler transforming target : {target} with scaler : {scaler_transforms[target]}')
    scaler = scaler_transforms[target]
    if scaler is not None:
        valid_pred[:, i] = scaler.inverse_transform(valid_pred[:, i].reshape(-1, 1)).flatten()


for i, target in enumerate(mean_columns):
    log_base = log_transforms[target]
    if log_base is not None and log_base != 'sqrt' and log_base != 'cbrt':
        valid_pred[:, i] = np.power(log_base, valid_pred[:, i])
    elif log_base == 'sqrt':
        valid_pred[:, i] = np.square(valid_pred[:, i])
    elif log_base == 'cbrt':
        valid_pred[:, i] = np.power(valid_pred[:, i], 3)

R2_valid = r2_score(y_valid, valid_pred)
MSE_valid = mean_squared_error(y_valid, valid_pred)
MAE_valid = mean_absolute_error(y_valid, valid_pred)

print(f'Valid scores:\nR2 : {R2_valid:.5f}, MSE : {MSE_valid:.5f}, MAE : {MAE_valid:.5f}')



2024-04-17 23:46:15.788967: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [9833,81]
	 [[{{node Placeholder/_1}}]]
2024-04-17 23:46:15.789290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [9833,6]
	 [[{{node Placeholder/_2}}]]


154/154 [==============================] - 159s 1s/step
Scaler transforming target : X4_mean with scaler : PowerTransformer()
Scaler transforming target : X11_mean with scaler : StandardScaler()
Scaler transforming target : X18_mean with scaler : None
Scaler transforming target : X50_mean with scaler : None
Scaler transforming target : X26_mean with scaler : MinMaxScaler()
Scaler transforming target : X3112_mean with scaler : None
Valid scores:
R2 : 0.30615, MSE : 639503.31029, MAE : 173.09033
